In [1]:
import py_core as py
import threading
import time
import ipywidgets as widgets
from IPython.display import display
from cpp_core import *
from ctypes import c_bool
from random import choice

cpp = load_cpp_scripts()

In [2]:
class Chomp:
    def __init__(self):
        self.evidence = cpp.ExtendedEvidence(cpp.IteratorCode.ZIGZAG)
        self.board = None
        self.flipped = False
chomp_object = Chomp()
player = True
waiting = False
null_proof = create_null("Proof")

def delay_action(delay, callback):
    def delayed_action():
        time.sleep(delay)
        callback()
    thread = threading.Thread(target=delayed_action)
    thread.start()

In [3]:
input_text = widgets.Text(
    placeholder="Try me: (7,7,7)"
)

turn_label = widgets.Label(
    value="Turn: ",
    layout=widgets.Layout(width="200px")
)
    
start_button = widgets.Button(
    description="Start",
    style=widgets.ButtonStyle(button_color="white"),
    layout=widgets.Layout(border="2px solid black")
)
def on_start_click(btn):
    global player, waiting
    waiting = True
    data = eval(input_text.value)
    board = py.Board(data)
    chomp_object.board = cpp.Board.get_board(board.as_ideal())
    chomp_object.flipped = board.flipped
    init_board(*data)
    if against_bot_checkbox.value:
        player = not bot_plays_first_checkbox.value
        waiting = not player
        turn_label.value = "Turn: "+ ("player" if player else "bot")
        if not player:
            delay_action(2, bot_action)
    else:
        waiting = False
        player = True
        turn_label.value = "Turn: first player"

start_button.on_click(on_start_click)

board_container = widgets.VBox()

def on_chomp_closure(row, col, _row):
    def on_chomp_click(btn):
        global player, waiting
        if waiting: return
        for i in range(_row):
            children = board_container.children[i].children
            board_container.children[i].children = children[:col]
        move = (col, row) if not chomp_object.flipped else (row, col)
        is_flipped = c_bool(False)
        chomp_object.board = chomp_object.board.chomp(move, is_flipped)
        if is_flipped: chomp_object.flipped = not chomp_object.flipped
        if against_bot_checkbox.value:
            player = not player
            waiting = not player
            if not player:
                delay_action(2, bot_action)
            turn_label.value = "Turn: " + ("player" if player else "bot")
        else:
            player = not player
            turn_label.value = "Turn: " + ("first player" if player else "second player")
    return on_chomp_click

def bot_action():
    global waiting
    proof = chomp_object.evidence.get(chomp_object.board)
    x, y = 0, 0
    if proof != null_proof:
        x, y = proof.move
    else: 
        moves = [move for move in cpp_iterator(cpp.Iterator(chomp_object.board))]
        if len(moves) == 0: return
        x, y = choice(moves)
    if chomp_object.flipped: x, y = y, x
    y = len(board_container.children) - y - 1
    waiting = False
    board_container.children[y].children[x].click()


def init_board(*board):
    rows = []
    number_of_rows = len(board)
    for row, number_of_columns in enumerate(board):
        children = []
        for col in range(number_of_columns):
            button = widgets.Button(
                description=" ",
                style=widgets.ButtonStyle(button_color="white", font_weight="bold"),
                layout=widgets.Layout(
                    border="2px solid black",
                    width="50px",
                    height="50px",
                    padding="2px",
                ),
            )
            if col == row == 0:
                button.style.button_color="red"
            else:
                button.on_click(on_chomp_closure(row, col, number_of_rows - row))
            children.append(button)
        rows.append(widgets.HBox(
            children=children,
            layout=widgets.Layout(
                height="54px",
            )
        ))
    board_container.children = tuple(rows[::-1])

against_bot_checkbox = widgets.Checkbox(
    value=False,
    description='Against Bot',
    disabled=False,
    indent=False
)

bot_plays_first_checkbox = widgets.Checkbox(
    value=False,
    description='Bot Plays First',
    disabled=True,
    indent=False
)

def on_against_bot_changed(change):
    if change['new']: 
        bot_plays_first_checkbox.disabled = False  
    else:
        bot_plays_first_checkbox.disabled = True 
        bot_plays_first_checkbox.value = False 

against_bot_checkbox.observe(on_against_bot_changed, names='value')


control = widgets.HBox([
    input_text,
    start_button,
    turn_label,
    against_bot_checkbox,
    bot_plays_first_checkbox
])
main = widgets.VBox([
    board_container,
    control
])

In [4]:
display(main)